In [22]:
import json
import os
import autogen
from autogen import ConversableAgent

class Base:
    # set API and keyu
    os.environ["OPENAI_API_BASE"] = "https://a.fe8.cn/v1"
    os.environ["OPENAI_API_KEY"] = "sk-jxGxEstKGNXlwD4gaFIIveFQOAWm7hRxUZth8k191o5m8DE4"

    def __init__(self):
        self.groupchat = None
        self.user = None
        self.requirement_file = None
        self.config_list = None
        self.groupt_chat_manager = None

    def getUser(self):
        self.user = MyConversableAgent(
            file_path=self.requirement_file,
            name = "User",
            llm_config = False,
            human_input_mode='ALWAYS',
            description='A User to provide the requirements.'
        )
        return self.user

    def getConfigList(self):
        self.config_list = [
            {
                "model": "gpt-4o",
                "api_key": os.environ.get("OPENAI_API_KEY"),
                "base_url": os.environ.get("OPENAI_API_BASE"),
            },
        ]
        return self.config_list

    def init_group_chat_manager(self):
        self.groupt_chat_manager = autogen.GroupChatManager(
            self.groupchat,
            llm_config={
                "temperature": 0.5,
                "config_list": self.config_list,
            }
        )

    def storeJSON(self, chat_results, agent_name, file_name, tag_name):
        content = [item['content'] for item in chat_results.chat_history if item.get('name') == agent_name]
        temp = "{\"" + tag_name+ "\": []}"
        data = json.loads(temp)
        with open(file_name, 'w') as f:
            json.dump(data, f, indent=4)
        for item in content:
            try:
                parsed_json = json.loads(item)
                if type(parsed_json) is list:
                    for sub_json in parsed_json:
                        data[tag_name].append(sub_json)
                else:
                    data[tag_name].append(parsed_json)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON: {e}")
        with open(file_name, 'w') as file:
            json.dump(data, file, indent=4)

class MyConversableAgent(ConversableAgent):
    def __init__(self, file_path, *args, **kwargs, ):
        super().__init__(*args, **kwargs)
        self.file_path = file_path  # Path to your input file
        self.current_index = 0  # Track the current index of req array
        with open(self.file_path, 'r', encoding="utf-8") as file:
            self.data = json.load(file)['requirements']

    def get_human_input(self, prompt: str) -> str:
        """
        Override to customize the way to get human input.
        For example, read input from a JSON file.
        Args:
            prompt (str): prompt for the human input.
        Returns:
            str: human input from a JSON file.
        """
        if self.current_index < len(self.data):
            # Get the current item from req array
            item = self.data[self.current_index]
            # Convert the item to string format
            reply = json.dumps(item)
            self.current_index += 1
        else:
            reply = "exit"  # Or any other suitable message
        self._human_input.append(reply)
        return reply

In [23]:
import datetime

import autogen
from autogen import ConversableAgent



class Term_Extractor(Base):
    def __init__(self):
        #system description
        with open('assets/system_description.txt', 'r', encoding='utf-8') as f:
            self.system_description = f.read()

        #Few-shot example
        with open('assets/few_shot_example_term_extraction.txt', 'r', encoding='utf-8') as I:
            self.few_shot_example = I.read()

        #chain of thought example
        with open('assets/chain_of_thought_term_extraction.txt', 'r') as f:
            self.chain_of_thought_term_extraction = f.read()

        # chain of thought example
        with open('assets/chain_of_thought_term_checker.txt', 'r') as f:
            self.chain_of_thought_term_checker = f.read()

        self.requirement_file = "assets/requirement_architecture.json"
        self.user = super().getUser()

        #term extraction agent
        self.term_extractor = ConversableAgent(
            name = "Term_Extractor",
            llm_config={
                "temperature": 0.5,
                "config_list": super().getConfigList(),
                "timeout": 600000,
                "cache_seed": None,
            },
            system_message =
            'You are an expert in requirement engineering.' +
            'Requirement statements can be categorised into Functional Requirements and Non-Functional Requirements. ' +
            'You will be provided with some requirement statements from User, which are for a system to be developed.' +
            'The description of the system is here: ' + self.system_description +
            'Your task is to extract Concepts and their Instances from the requirement statements.' +
            'The requirements will be provided by User, please note that the requirements are not ordered, and one requirement may implicitly depend on other requirements. ' +
            'Each requirement statement is identified by a unique GID (short for Global ID). ' +
            'Please extract occurrences of the following from the requirement statements: "GID", "Concepts" and "Instances". ' +
            'The rationale for your task is here: ' + self.chain_of_thought_term_extraction +
            'The Output format should be in JSON only, no markdown permitted (no ```), no explain.' +
            'Include only contents provided to you.',
            description="An Term_Extractor to extract instances of a sentence."
        )
        #Term JSON checker
        self.term_checker = ConversableAgent(
            name="Term_checker",
            llm_config={
                "temperature": 0.5,
                "config_list": super().getConfigList(),
                "timeout": 600000,
                "cache_seed": None,
            },
           # llm_config=llm_default_config,
            system_message=
            'You are an expert in requirement engineering and Model Driven Engineering (MDE). ' +
            'In MDE, engineers produce "Domain Specific Models (DSLs)" or "metamodels". ' +
            'A metamodel defines the abstract syntax of a modelling language, which may contain: 1) Classes to describe the concepts in the target system (to be developed); ' +
            '2) Attributes with primitive types inside the Classes; and 3) References (either association or containment) among Classes. ' +
            'Each concept defined in a metamodel is also referred to as a "meta-element". ' +
            'Once a metamodel is in place, engineers can use it to create "instance models" to model their systems, which conform to the abstract syntax defined in the metamodel. ' +
            'Each instance of a "meta-element" in the instance model may also be referred to as an "instance object" of that "meta-element". ' +
            'In order to extract DSLs, one need to make sure the Concepts and their Instances are correctly extracted. ' +
            'Your task is to check whether the Concepts and Instances extracted by Term_Extractor are correct. ' +
            'To provide you with context, the description of the system to be developed is here: ' + self.system_description +
            'The rationale for your task is here: ' + self.chain_of_thought_term_checker +
            'Here are some examples of the extracted models, which may help you.' + self.few_shot_example +
            'The Output format should be in JSON only, no markdown permitted (no ```), no explain.' +
            'Include only contents provided to you and Term_Extractor.',
            description="A Term_checker to check the Output format from Term_Extractor."
        )

        self.groupchat = autogen.GroupChat(
            agents=[self.user, self.term_extractor, self.term_checker],
            messages=[],
            speaker_selection_method='round_robin',
            max_round=100,
            send_introductions=True
        )

        super().init_group_chat_manager()

    def run_extraction_multiple_times(self, times):
        for i in range(times):
            term_extractor = Term_Extractor()

            chat_results = term_extractor.user.initiate_chat(
                term_extractor.groupt_chat_manager,
                # modify here to start your chat
                message="",
                clear_history=True,
            )
            result_file = f"result/Term_extraction_result/result_term_extractor_{i + 1}.json"
            term_extractor.storeJSON(chat_results, "Term_checker", result_file,"term_trace")

'import datetime\n\nimport autogen\nfrom autogen import ConversableAgent\n\n\n\nclass Term_Extractor(Base):\n    def __init__(self):\n        #system description\n        with open(\'assets/system_description.txt\', \'r\', encoding=\'utf-8\') as f:\n            self.system_description = f.read()\n\n        #Few-shot example\n        with open(\'assets/few_shot_example_term_extraction.txt\', \'r\', encoding=\'utf-8\') as I:\n            self.few_shot_example = I.read()\n\n        #chain of thought example\n        with open(\'assets/chain_of_thought_term_extraction.txt\', \'r\') as f:\n            self.chain_of_thought_term_extraction = f.read()\n\n        # chain of thought example\n        with open(\'assets/chain_of_thought_term_checker.txt\', \'r\') as f:\n            self.chain_of_thought_term_checker = f.read()\n\n        self.requirement_file = "assets/requirement_architecture.json"\n        self.user = super().getUser()\n\n        #term extraction agent\n        self.term_extrac

In [24]:
Term_Extractor().run_extraction_multiple_times(2)

'Term_Extractor().run_extraction_multiple_times(2)'

In [25]:
class DSL_Extractor(Base): 
    def run_dsl_extraction(times):
        for i in range(times):    
                def __init__(self):
                    #system description
                    with open('assets/system_description.txt', 'r', encoding='utf-8') as system_file:
                        system_description = system_file.read()
            
                    #Few-shot example for dsl extraction
                    with open('assets/few_shot_example_dsl_extraction.txt', 'r', encoding='utf-8') as few_shot_dsl_file:
                        few_shot_example_dsl_extraction = few_shot_dsl_file.read()
            
                    #Few-shot example for dsl checker
                    with open('assets/few_shot_example_dsl_checker.txt', 'r', encoding='utf-8') as few_shot_dsl_checker:
                        few_shot_example_dsl_checker = few_shot_dsl_checker.read()
            
                    #chain of thought prompt for dsl extraction
                    with open('assets/chain_of_thought_dsl_extraction.txt', 'r') as cot_dsl_extraction:
                        chain_of_thought_dsl_extraction = cot_dsl_extraction.read()
            
                    #chain of thought prompt for dsl checker
                    with open('assets/chain_of_thought_dsl_checker.txt', 'r') as cot_dsl_checker:
                        chain_of_thought_dsl_checker = cot_dsl_checker.read()
            
                    #Extracted terms
                    with open(f"result/Term_extraction_result/result_term_extractor_{i + 1}.json", 'r', encoding='utf-8') as term_file:
                        extracted_terms = term_file.read()
            
                    #Emfatic rules
                    with open('assets/emfatic.txt', 'r', encoding='utf-8') as emfatic_rules_file:
                        emfatic_rules = emfatic_rules_file.read()
            
                    self.requirement_file = "assets/requirement_architecture.json"
                    self.user = super().getUser()
            
                    #DSL Extractor to create emfatic code
                    self.dsl_extractor = ConversableAgent(
                        name="DSL_Extractor",
                        llm_config={
                            "temperature": 0.5,
                            "config_list": super().getConfigList(),
                            "timeout": 600000,
                            "cache_seed": None,
                        },
                        system_message=
                        'You are an expert in Model Driven Engineering (MDE). ' +
                        'You are experienced in creating "Domain Specific Languages (DSLs)" or "metamodels". ' +
                        'A metamodel contains the abstract syntax to capture the concepts for the system to be developed. ' +
                        'You have extensive experience in creating metamodels using the Ecore modeling language provided by the Eclipse Modeling Framework (EMF). ' +
                        'You are also an expert in creating metamodels using the Emfatic language (a textual syntax for Ecore) to create metamodels.' +
                        'Your job is to create a metamodel written in Emfatic based on the requirements provided to you in JSON every round. ' +
                        'The description of the system to be developed is here: ' + system_description +
                        'Previously, a Term model is extracted, in here: ' + extracted_terms +
                        'Here are some thinking processes to guide you: ' + chain_of_thought_dsl_extraction +
                        'Here are some examples of the extracted metamodels based on inputs:' + few_shot_example_dsl_extraction +
                        'Consider also the previous inputs from the User. ' +
                        'You must only include Emfatic code in your responses.' +
                        'Do not include any other contents that are not provided to you. ' +
                        'Please do not include explanations in your answers. In addition, no markdown is permitted (no ```). ',
                        description='A DSL extractor to create metamodels using Emfatic based on the input and responses from the User.'
                    )
            
                    #DSL checker to check the syntax of Emfatic code
                    self.dsl_checker = ConversableAgent(
                        name="DSL_Checker",
                        llm_config={
                            "temperature": 0.5,
                            "config_list": super().getConfigList(),
                            "timeout": 600000,
                            "cache_seed": None,
                        },
                        system_message=
                        'You are an expert in Model Driven Engineering (MDE). ' +
                        'Specifically, you are experienced in create "Domain Specific Languages (DSLs)" or "metamodels". ' +
                        'A metamodel contains the abstract syntax to capture the concepts for the system to be developed. ' +
                        'You have extensive experience in creating metamodels using the Ecore modeling language provided by the Eclipse Modeling Framework (EMF). '
                        'You are also an expert in using the Emfatic language (a textual syntax for Ecore) to create metamodels. ' +
                        'An description of the language syntax for Emfatic is here: ' + emfatic_rules +
                        'Your job is to check whether the metamodel provided to you written in Emfatic is correct, in every round of conversation. ' +
                        'You should also consult the thinking process in here: ' + chain_of_thought_dsl_checker +
                        'Here are some examples of the extracted metamodels based on inputs:' + few_shot_example_dsl_checker +
                        'You should also consult the terms extracted previously (but do not rely on it completely): ' + extracted_terms +
                        'You must only include Emfatic code in your responses.' +
                        'Do not include any other contents that are not provided to you. ' +
                        'Please do not include explanations in your answers. In addition, no markdown is permitted (no ```). ',
                        description='A Model_Checker to check the syntax of Emfatic code.'
                    )
            
                    #DSL refactor to summerize the code from Model_Checker&Modeller
                    self.dsl_refactorer = ConversableAgent(
                        name="DSL_Refactor",
                        llm_config={
                            "temperature": 0.5,
                            "config_list": super().getConfigList(),
                            "timeout": 600000,
                            "cache_seed": None,
                        },
                        system_message=
                        'You are an expert in Model Driven Engineering (MDE), and you are working with some experts in creating "Domain Specific Languages (DSLs)" or "metamodels". ' +
                        'DSL_Extractor has created the DSL using Emfatic based on the inputs from User. ' +
                        'Model_Checker has checked the syntax of the Emfatic code. ' +
                        'Your job is to organise and refactor (when necessary) the Emfatic code provided by Model_Checker. ' +
                        'In every round, you provide your answer, you need to combine with your previous answer and update them in one package. ' +
                        'Pay attention to containement and non-containment references. ' +
                        'In the requirement, when it says "A contains B" or "A defines B", it often denotes a containment reference. ' +
                        'In the requirement, when it says "A uses B" or "A refers to B", it often denotes a non-containment reference. ' +
                        'Refactor references when necessary by looking at the requirements as a whole. ' +
                        'Your answer needs to be correctly written in Emfatic. ' +
                        'You must include only Emfatic code in your responses.' +
                        'Do not include any other contents that are not provided to you. ' +
                        'Please do not explain. In addition, no markdown is permitted (no ```)',
                        description='A DSL_Refactor to organise and refactor the code from Model_Checker.'
                    )
            
                    #Json_Generator to generate a json for traceability.
                    self.json_generator = ConversableAgent(
                        name="Json_Generator",
                        llm_config={
                            "temperature": 0.5,
                            "config_list": super().getConfigList(),
                            "timeout": 600000,
                            "cache_seed": None,
                        },
                        system_message=
                        'You are an expert in Model Driven Engineering (MDE). ' +
                        'Your job is to generate a traceability JSON element based on the Emfatic code provided by DSL_Summariser and the requirement in JSON provided by User every round. ' +
                        'In the Emfatic code provided to you by DSL_Summariser, you shall identify which class, attributes, references and operations are created, based on the requirement JSON provided by User' +
                        'Your answer should be in the following JSON format : ' +
                        '{"requirement_gid" : ${the gid of the requirement provided by User}, ' +
                        '"Emfatic_class" : ${the class name provided by Model_Checker}, ' +
                        '"Emfatic_attributes": ${the "attr"s that the class contains (if not null)}, ' +
                        '"Emfatic_references":${the "ref"s or "val"s that the class contains (if not null)}, ' +
                        '"Emfatic_operations": ${the "op"s that class contains (if not null)} }' +
                        'If the Emfatic code creates multiple classes, please create a JSON list with multiple JSON entries, like so: [$JSON entry, $JSON entry, ... $JSON entry].' +
                        ' The Output format is JSON only, no markdown permitted (no ```), no explain.',
                        description='A Json_Generator to generate a json element based on the Emfatic code provided by Model Checker and the requirement provided by User every round.'
                    )
            
                    #Create a groupchat to organise the agents
                    self.groupchat = autogen.GroupChat(
                        agents=[self.user, self.dsl_extractor, self.dsl_checker, self.dsl_refactorer, self.json_generator],
                        messages=[],
                        speaker_selection_method='round_robin',
                        max_round=500,
                        send_introductions=True
                    )
            
                    super().init_group_chat_manager()
                    dsl_extractor = DSL_Extractor()
                    chat_results = dsl_extractor.user.initiate_chat(
                        dsl_extractor.groupt_chat_manager,
                        # message="Let us start extracting Domain Specific Languages (DSL) or metamodels from the requirements in JSON. " +
                        # "DSL_Extractor will extract the DSL, DSL_Checker will check the correctness of the DSL, DSL_Refactor will organise and refactor the DSL. " +
                        # "The JSON_Generator will generate the JSON file containing trace information.",
                        clear_history=True,
                    )
        
                    dsl_extractor.storeJSON(chat_results, "Json_Generator", f"result/DSL_trace_result/multi_dsl_trace_result_{i+1}.json", "dsl_trace")
        
                    data = chat_results.chat_history
        
                    # Extract the last content for 'DSL_Refactor'
                    last_dsl_refactor_content = next(
                        (entry['content'] for entry in reversed(data) if entry.get('name') == 'DSL_Refactor'), None)
        
                    if last_dsl_refactor_content:
                        with open(f'result/DSL_extraction_result/multi_DSL_extraction_result_{i+1}.emf', 'w', encoding='utf-8') as file:
                            file.write(last_dsl_refactor_content)
        
                    # Print the last content
                    print(last_dsl_refactor_content)


'class DSL_Extractor(Base): \n    def run_dsl_extraction(times):\n        for i in range(times):    \n                def __init__(self):\n                    #system description\n                    with open(\'assets/system_description.txt\', \'r\', encoding=\'utf-8\') as system_file:\n                        system_description = system_file.read()\n            \n                    #Few-shot example for dsl extraction\n                    with open(\'assets/few_shot_example_dsl_extraction.txt\', \'r\', encoding=\'utf-8\') as few_shot_dsl_file:\n                        few_shot_example_dsl_extraction = few_shot_dsl_file.read()\n            \n                    #Few-shot example for dsl checker\n                    with open(\'assets/few_shot_example_dsl_checker.txt\', \'r\', encoding=\'utf-8\') as few_shot_dsl_checker:\n                        few_shot_example_dsl_checker = few_shot_dsl_checker.read()\n            \n                    #chain of thought prompt for dsl extraction\n  

In [ ]:
def EMF_Model_Creation(times):
    for i in range(times):
        class Model_Creator(Base):
            def __init__(self,i):
                #system description
                with open('assets/system_description.txt', 'r', encoding='utf-8') as f:
                    system_description = f.read()
        
                #Few-shot example for modeller
                with open('assets/few_shot_example_model_creation.txt', 'r', encoding='utf-8') as f:
                    few_shot_example_model_creation = f.read()
        
                #Few-shot example for model checker
                with open('assets/few_shot_example_model_checker.txt', 'r', encoding='utf-8') as f:
                    few_shot_example_model_checker = f.read()
        
                #chain of thought prompt for modeller
                with open('assets/chain_of_thought_model_creation.txt', 'r') as f:
                    chain_of_thought_model_creation = f.read()
        
                #chain of thought prompt for model checker
                with open('assets/chain_of_thought_model_checker.txt', 'r') as f:
                    chain_of_thought_model_checker = f.read()
        
                #chain of thought prompt for model checker
                with open('assets/chain_of_thought_model_trace.txt', 'r') as f:
                    chain_of_thought_model_trace = f.read()
        
                #chain of thought prompt for model checker
                with open(f"result/Term_extraction_result/result_term_extractor_{i + 1}.json", 'r') as f:
                    term_extraction = f.read()
        
                #Emfatic rules
                with open(f'result/DSL_extraction_result/multi_DSL_extraction_result_{i+1}.emf', 'r', encoding='utf-8') as DSL_source:
                    dsl = DSL_source.read()
        
                self.requirement_file = "assets/requirement_architecture.json"
                self.user = super().getUser()
        
                #Modeller to create emfatic code
                self.model_Creator = ConversableAgent(
                    name="Model_Creator",
                    llm_config={
                        "temperature": 0.5,
                        "config_list": super().getConfigList(),
                        "timeout": 600000,
                        "cache_seed": None,
                    },
                    system_message=
                    'You are an expert in Model Driven Engineering (MDE). ' +
                    'You are experienced in creating EMF (Eclipse Modelling Framework) models using the Epsilon Object Language (EOL). ' +
                    'Your job is to write an EOL program to create an EMF model that conforms to the EMF metamodel (written in the Emfatic language) in here: ' + dsl +
                    'You should base your answer on the requirements provided to you by User. ' +
                    #'The description of the system to be developed is here: ' + system_description +
                    'The Concepts extracted (in JSON) from the requirements are here: ' + term_extraction +
                    'Consult the thinking process here: ' + chain_of_thought_model_creation +
                    'Here are some examples based on input requirements:' + few_shot_example_model_creation +
                    'You should also consult the code produced by Model_Refactor. ' +
                    'You must only include EOL code in your responses. ' +
                    'Do not include any other contents that are not provided to you. ' +
                    'Please do not include explanations in your answers. Do not include markdowns (no ```). ',
                    description='A EMF model creator using EOL, based on the inputs and responses from the User.'
                )
        
                #Model_Checker to check the syntax of Emfatic code
                self.model_Checker = ConversableAgent(
                    name="Model_Checker",
                    llm_config={
                        "temperature": 0.5,
                        "config_list": super().getConfigList(),
                        "timeout": 600000,
                        "cache_seed": None,
                    },
                    system_message=
                    'You are an expert in Model Driven Engineering (MDE). ' +
                    'You are experienced in creating EMF (Eclipse Modelling Framework) models using the Epsilon Object Language (EOL). ' +
                    'Your job is to check if the EOL program provided to you is correct, and respond with the correct EOL code.' +
                    'You should base your answer on both the EMF metamodel (written in Emfatic) here:'  + dsl +
                    ', and the requirements provided by User. ' +
                    'The description of the system to be developed is here: ' + system_description +
                    'The Concepts extracted (in JSON) from the requirements are here: ' + term_extraction +
                    'Consult the thinking process in here: ' + chain_of_thought_model_checker +
                    'Here are some examples of the extracted metamodels based on inputs:' + few_shot_example_model_checker +
                    #'You should also consult the code produced by Model_Refactor. ' +
                    'You must only include EOL code in your responses.' +
                    'Do not include any other contents that are not provided to you. ' +
                    #'If no further answer can be provided, repeat your last answer. ' +
                    'Please do not include explanations in your answers. In addition, no markdown is permitted (no ```). ',
                    description='A program checker to check the syntax of the EOL code.'
                )
        
                #Model_Summerizer to summerize the code from Model_Checker&Modeller
                self.model_Refactorer = ConversableAgent(
                    name="Model_Refactorer",
                    llm_config={
                        "temperature": 0.5,
                        "config_list": super().getConfigList(),
                        "timeout": 600000,
                        "cache_seed": None,
                    },
                    system_message=
                    'You are an expert in Model Driven Engineering (MDE), and you are working with some experts in creating models that conform to a metamodel written in the Emfatic language. ' +
                    'Model_Creator has created the source code in Epsilon Object Language (EOL) based on the inputs from User. ' +
                    'Model_Checker has checked the syntax of the EOL code. ' +
                    'Your job is to organise and refactor the EOL code (when necessary) provided by them. ' +
                    'In every round, you provide your answer, you need to combine with your previous answer and update them. ' +
                    'Your answer needs to be correctly written in EOL. ' +
                    'Do not include any other contents that are not provided to you. ' +
                    'If no further answer can be provided, repeat your last answer. ' +
                    'Please only provide EOL code in your answers. Do not explain (no ```). ',
                    description='A Model_Summariser to summerise the code from Model_Checker.'
                )
        
                #Json_Generator to generate a json for traceability.
                self.json_generator = ConversableAgent(
                    name="Json_Generator",
                    llm_config={
                        "temperature": 0.5,
                        "config_list": super().getConfigList(),
                        "timeout": 600000,
                        "cache_seed": None,
                    },
                    system_message=
                    'You are an expert in Model Driven Engineering (MDE). ' +
                    'You are experienced in creating EMF (Eclipse Modelling Framework) models using languages provided by the Eclipse Epsilon platform. ' +
                    'Specifically, you use the Epsilon Object Language (EOL) to create EMF models programmatically. ' +
                    'Your job is to generate a JSON element by analysing the EOL code (provided by Model_Checker) for model creation, and the requirement provided by User every round. ' +
                    'The metamodel (created using the Emfatic language) for the model to be created is here: ' + dsl +
                    'Here are some guidance to help you: ' + chain_of_thought_model_trace +
                    'Your answer should be in the following JSON format : ' +
                    '{"requirement_gid" : ${the gid of the requirement provided by User}, ' +
                    '"Model_Element" : ${the name of the class created by the EOL code}, ' +
                    '"Model_Element_id" : ${the name or the id of the object created for the above class}, ' +
                    '"Model_attribute": ${the name of the attributes set by the EOL code}, ' +
                    '"Attribute_value": ${the value of the above attribute set by the EOL code}, ' +
                    '"Model_references":${the name of the references set by the EOL code} } ' +
                    '"Reference_value":${the name or the id of the value set for the above reference} } ' +
                    'If the EOL code creates multiple model elements, please create a JSON list with multiple JSON entries, like so: [$JSON entry, $JSON entry, ... $JSON entry].' +
                    ' The Output format is JSON only, no markdown permitted (no ```), no explain.',
                    description='A Json_Generator to generate a JSON element based on the EOL code provided by Model_Checker and the requirement provided by User every round.'
                )       
                #Create a groupchat to organise the agents
                self.groupchat = autogen.GroupChat(
                    agents=[self.user, self.model_Creator, self.model_Checker, self.model_Refactorer, self.json_generator],
                    messages=[],
                    speaker_selection_method='round_robin',
                    max_round=1000,
                    send_introductions=True
                )
                super().init_group_chat_manager()
                model_creator = Model_Creator()
                chat_results = model_creator.user.initiate_chat(
                    model_creator.groupt_chat_manager,
                    # start chat with the first item of the file
                    # message="Let us start creating the EOL code for creating a EMF model based on the requirements provided by User. " +
                    # "Model_Creator will write the EOL code, Model_Checker will check the correctness of the code, Model_Refactorer will organise and refactor the code. " +
                    # "The JSON_Generator will generate the JSON file containing trace information.",
                    clear_history=True,
                )
    
                model_creator.storeJSON(chat_results, "Json_Generator", f"result/EMF_Model_trace_result/multi_EOL_trace_result_{i+1}.json", "model_trace")
    
                data = chat_results.chat_history
    
                # Extract the last content for 'Model_Summariser'
                last_model_summariser_content = next(
                    (entry['content'] for entry in reversed(data) if entry.get('name') == 'Model_Refactorer'), None)
    
                if last_model_summariser_content:
                    with open(f'result/EMF_Model_result/multi_eol_result_{i+1}.eol', 'w', encoding='utf-8') as file:
                        file.write(last_model_summariser_content)
                # Print the last content
                print(last_model_summariser_content)

In [ ]:
EMF_Model_Creation(2)